In [0]:
#importing of necessary libraries and modules (recommended to run on Google colab)

!pip install mpnum
import numpy as np
import pickle

from google.colab import drive
drive.mount('/content/gdrive/')

'''we need to import TFI_processing,
thus, we go to the VAE_for_quantum_data
directory where the file is situated'''
%cd gdrive
%cd My Drive
%cd VAE_for_quantum_data

import TFI_processing as ising

# Data generation (measurement outcomes)

In [0]:
#spin chain length
N = 32

#batch size
SamplesPerBatch = 100000
#number of batches
NofBatches = 5
#number of fields
NofFields = 21
#initial magn. field
FieldMin = 0.
#final magn. field
FieldMax = 2.
#step along field
dField   = (FieldMax-FieldMin)/(NofFields-1)

import time

start_time = time.time()
for f in range(0, NofFields):
    ta    = FieldMin + dField*f
    model = ising.ising_chain(N, ta, la=0.00001)
    model.gs()
    
    filehandler = open("data/model_N=" + str(N) + "_h=" + str(round(ta, 4)) + ".obj", 'wb') 
    pickle.dump(model, filehandler)
    
    if f==0:
        samples_h = np.ones(NofBatches * SamplesPerBatch)*ta
    else:
        samples_h = np.append(samples_h, np.ones(NofBatches*SamplesPerBatch)*ta)
    print("---gs for field=", ta, "progress",f+1,"out of",NofFields)
    print("---time elapsed",time.time() - start_time)
    for i in range(0, NofBatches):
        print("   ---samples generation:", i+1, "out of", NofBatches)
        if (i==0) and (f==0):
            samples_x = model.sample(SamplesPerBatch)
        else:
            samples_x = np.append(samples_x, model.sample(SamplesPerBatch), axis=0)

# Data saving

In [0]:
np.save("data/X_values_N=" + str(N) + ".npy", samples_x)
np.save("data/H_values_N=" + str(N) + ".npy", samples_h)